# Download a MEA1k data set
This recording from a mouse? retina was done by [whom](http://pothal.com/wp-content/uploads/2016/08/unknown-user.jpg) at ETHZ?

In [ ]:
import urllib.request
file_url = 'https://datasync.ed.ac.uk/public.php/webdav'

print('Downloading data file - this may take a while, patience...')
password_mgr = urllib.request.HTTPPasswordMgrWithDefaultRealm()
password_mgr.add_password(None, file_url, "Bu5OJl7kfP3bBT4", "HS2")
handler = urllib.request.HTTPBasicAuthHandler(password_mgr)
opener = urllib.request.build_opener(handler)
opener.open(file_url)
urllib.request.install_opener(opener)
with urllib.request.urlopen(file_url) as response, open('mea1k_data.h5', 'wb') as out_file:
    data = response.read()
    out_file.write(data)
print('...done')

# Use of the `HSDetection` class

In [ ]:
from herdingspikes import HSDetection
from herdingspikes.probe import Mea1k

import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [ ]:
# raw data location
data_file = 'mea1k_data.h5'

# detection parameters
to_localize = True
cutout_start = 10
cutout_end = 30
threshold = 7
file_name = 'ProcessedSpikes_mea1k'

In [ ]:
Probe = Mea1k(data_file)
H = HSDetection(Probe, to_localize, cutout_start, cutout_end, threshold,
                maa=0, maxsl=12, minsl=3, ahpthr=0, out_file_name=file_name)

In [ ]:
Probe.show(figwidth=10)

In [ ]:
# plot some raw traces

t1,t2 = 0,5000
data = Probe.Read(t1,t2).reshape(t2-t1,Probe.num_channels)
plt.figure(figsize=(16,4))
ch = 67
for ch in range(60,68):
    plt.plot(data[4000:4600,ch])

In [ ]:
H.DetectFromRaw()

In [ ]:
H.LoadDetected()

In [ ]:
plt.figure(figsize=(10, 10))
H.PlotTracesChannels(0)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(5, 5))
H.PlotAll(invert=True, s=2, alpha=1)
plt.axis('equal');

# Use of the `HSClustering` class

In [ ]:
from herdingspikes import HSClustering
from sklearn.cluster import DBSCAN

In [ ]:
# Load from file
#C = HSClustering(['../data/ProcessedSpikes_mea1k.bin'], cutout_length=41)

# Or if the spikes are already in memory
# simply load from the Detection class
C = HSClustering(H)

In [ ]:
C.ShapePCA(pca_ncomponents=2, pca_whiten=True)
C.CombinedClustering(alpha=5, bandwidth=5, bin_seeding=False, n_jobs=-1)

In [ ]:
plt.figure(figsize=(8, 8))
ax = plt.subplot(111)
C.PlotAll(invert=True, s=1, show_labels=True)
plt.title("MEan Shift")


In [ ]:
units = range(11)
C.PlotShapes(units)

In [ ]:
C.CombinedClustering(eps=1., alpha=1.5, clustering_algorithm=DBSCAN,
                    min_samples=5, n_jobs=-1)

In [ ]:
plt.figure(figsize=(8, 8))
C.PlotAll(invert=True, s=9,alpha=0.5)
plt.title("DBSCAN")


In [ ]:
units = range(6)
C.PlotShapes(units)

In [ ]:
C.SaveHDF5(["mea1k_sorted.hdf5"])